In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

# Time to Search

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print(item.properties)

## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [ ]:
from weaviate.classes.query import Filter
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5,
    filters=Filter.by_property("category").like("COMMON")
)

for item in response.objects:
    print(item.properties)

## Keyword Search
[Docs - bm25](https://weaviate.io/developers/weaviate/search/bm25)

In [ ]:
response = jeopardy.query.bm25(
    query="railroad",
    # query="railroad africa",
    limit=10,
)

for item in response.objects:
    print(item.properties)

In [ ]:
response = jeopardy.query.bm25(
    query="railroad africa",
    query_properties=["answer"],
    limit=10,
)

for item in response.objects:
    print(item.properties)

## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="rail, trains and cars",
    alpha=0.8,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

In [ ]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

In [ ]:
from weaviate.classes.query import MetadataQuery
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    return_metadata=MetadataQuery(explain_score=True),
    limit=5,
)

for item in response.objects:
    print(item.metadata.explain_score)
    print(item.properties["answer"], " || ", item.properties["question"])

## Close the client when done

In [ ]:
client.close()